## NTUEE儀表板資料處理中心 - 祝您好孕
##### 作者：NTUEE儀表板資料組

#### 引入必要套件與函數

In [18]:
%%capture 
!pip3 install pandas
!pip3 install geocoder
!pip3 install geopandas
!pip3 install shapely

##### 安裝套件、函數宣告


In [124]:
%%capture
import requests
import pandas as pd
import geocoder
import geopandas as gpd
from shapely.geometry import Point, Polygon
## 從data.taipei抓取資料
def get_datataipei_api(rid):
    url = f"""https://data.taipei/api/v1/dataset/{rid}?scope=resourceAquire"""
    response = requests.request("GET", url)
    data_dict = response.json()
    count = data_dict['result']['count']
    res = list()
    offset_count = int(count/1000)
    for i in range(offset_count+1):
        i = i* 1000
        url = f"""https://data.taipei/api/v1/dataset/{rid}?scope=resourceAquire&offset={i}&limit=1000"""
        response = requests.request("GET", url)
        get_json = response.json()
        res.extend(get_json['result']['results'])
    return pd.DataFrame(res)
## 用geocoder取得TWD97座標
def 新增經緯度TWD97座標行以及轉GeoDataFrame(資料, 地址欄位名稱):
	資料['geometry'] = 資料[地址欄位名稱].apply(
		lambda x: (
			lambda a: Point(a.json['lng'], a.json['lat']) if a and a.json else None
		)(geocoder.arcgis(x))
	)
	# 沒查到經緯度的話...郭囉 誰叫你選那麼奇怪的址
	資料.dropna(subset=['geometry'], inplace=True)
	資料 = gpd.GeoDataFrame(
		資料,
		geometry='geometry', 
		crs='EPSG:4326'
	)
	資料 = 資料.to_crs('EPSG:3826') # TWD97
	return 資料
## 印資料
def 印資料(資料):
	print("這份資料總共有", 資料.shape[0], "筆！")
	print(資料.head(資料.shape[0]))

##### 資料庫（不斷更新！）


In [20]:
祝您好孕資料庫 = {
    "112上半年婦幼安全警示地點": "016bf772-71bd-4500-aba2-22387a1d0ae7"
}

### 祝您好孕

##### 點子
1. 婦幼安全警示區分佈圖

##### 婦幼安全警示區分佈圖

In [125]:
# 取得資料
婦幼安全警示區資料 = get_datataipei_api(祝您好孕資料庫["112上半年婦幼安全警示地點"])
婦幼安全警示區資料.drop('_id', axis=1, inplace=True)
婦幼安全警示區資料.drop('_importdate', axis=1, inplace=True)
印資料(婦幼安全警示區資料)


這份資料總共有 20 筆！
                 編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註
0               1\t臺北市大同區捷運中山站\t大同分局\t103\t續列
1     2\t臺北市萬華區峨嵋街、漢中街口(捷運西門站)\t萬華分局\t108\t續列
2   3\t臺北市萬華區西園路、和平西路口(捷運龍山寺站)\t萬華分局\t108\t續列
3             4\t臺北市中山區捷運松江南京站\t中山分局\t104\t續列
4             5\t臺北市中山區捷運中山國小站\t中山分局\t104\t續列
5             6\t臺北市大安區捷運忠孝復興站\t大安分局\t106\t續列
6            7\t臺北市中正區捷運臺北車站\t中正第一分局\t100\t續列
7             8\t臺北市中正區二二八公園\t中正第一分局\t100\t續列
8             9\t臺北市中正區捷運古亭站\t中正第二分局\t100\t續列
9              10\t臺北市松山區新東街9巷\t松山分局\t105\t續列
10                            11\t"臺北市信義區松廉路段
11         (松仁路至松智路區塊內之廊道及空橋)"\t信義分局\t110\t續列
12     12\t臺北市信義區松壽路段(松智路至松仁路)\t信義分局\t110\t續列
13             13\t臺北市北投區捷運劍潭站\t士林分局\t111\t新增
14             14\t臺北市北投區捷運石牌站\t北投分局\t112\t續列
15     15\t臺北市文山區指南路道南橋下涵洞附近\t文山第一分局\t116\t續列
16         16\t臺北市文山區捷運萬芳醫院站\t文山第二分局\t116\t續列
17          17\t臺北市南港區捷運南港展覽館站\t南港分局\t115\t續列
18         18\t臺北市內湖區麥帥二橋下河濱公園\t內湖分局\t114\t續列
19         19\t臺北市內湖區成功路五段420巷\t內湖分局\t114\t新增


In [127]:
# 整理一下資料
婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'][10] += 婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'][11]
婦幼安全警示區資料.drop(11, inplace=True)
婦幼安全警示區資料.reset_index(drop=True, inplace=True)
婦幼安全警示區資料['地點位置'] = 婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'].str.split('\t').str[1]
婦幼安全警示區資料['管轄分局'] = 婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'].str.split('\t').str[2]
婦幼安全警示區資料.drop('編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註', axis=1, inplace=True)
印資料(婦幼安全警示區資料)

,地點位置,管轄分局
0,臺北市大同區捷運中山站,大同分局
1,臺北市萬華區峨嵋街、漢中街口(捷運西門站),萬華分局
2,臺北市萬華區西園路、和平西路口(捷運龍山寺站),萬華分局
3,臺北市中山區捷運松江南京站,中山分局
4,臺北市中山區捷運中山國小站,中山分局
5,臺北市大安區捷運忠孝復興站,大安分局
6,臺北市中正區捷運臺北車站,中正第一分局
7,臺北市中正區二二八公園,中正第一分局
8,臺北市中正區捷運古亭站,中正第二分局
9,臺北市松山區新東街9巷,松山分局


In [67]:
婦幼安全警示區資料GDF = 新增經緯度TWD97座標行以及轉GeoDataFrame(婦幼安全警示區資料, '地點位置')

,地點位置,管轄分局,geometry
0,臺北市大同區捷運中山站,大同分局,POINT (301998.139 2773179.209)
1,臺北市萬華區峨嵋街、漢中街口(捷運西門站),萬華分局,POINT (301155.709 2770469.962)
2,臺北市萬華區西園路、和平西路口(捷運龍山寺站),萬華分局,POINT (299863.095 2769828.402)
3,臺北市中山區捷運松江南京站,中山分局,POINT (303804.704 2773020.062)
4,臺北市中山區捷運中山國小站,中山分局,POINT (303151.639 2772836.951)
5,臺北市大安區捷運忠孝復興站,大安分局,POINT (304796.528 2770491.862)
6,臺北市中正區捷運臺北車站,中正第一分局,POINT (302044.290 2771130.192)
7,臺北市中正區二二八公園,中正第一分局,POINT (302330.141 2769474.211)
8,臺北市中正區捷運古亭站,中正第二分局,POINT (302330.141 2769474.211)
9,臺北市松山區新東街9巷,松山分局,POINT (307014.558 2772304.637)
